In [347]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


## Import and format data

In [285]:
df = pd.read_csv('SEG_Saleshistory_Stores.csv')
df.columns = ['date', 'store', 'item', 'sales']
df.date = pd.to_datetime(df.date)
df.item = df.item.astype('category')
df.store = df.store.astype('category')
df.dtypes

date     datetime64[ns]
store          category
item           category
sales             int64
dtype: object

Below are some summary statistics on the data. Overall the quantities for individual items at individual stores is quite small. It would be difficult to forecast daily quantities of individual items and individual stores, so we will work towards forecasting weekly item sales at individual stores.

In [237]:
df.describe(include='all')

,date,store,item,sales
count,425575,425575,425575.0,425575.000000
unique,342,181,39.0,NaN
top,2016-11-05 00:00:00,SEGWD7,41795.0,NaN
freq,4110,6111,27516.0,NaN
first,2016-02-04 00:00:00,NaN,NaN,NaN
last,2017-01-11 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.670345
std,NaN,NaN,NaN,1.112165
min,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,1.000000


Let's take a look at a few cuts on the data to see if we can spot any trends. Below are plots of a handful of individual stores' sales. It looks like the answer to the question about the big jump in sales in September is the addition of a good number of stores.

### Total Sales

In [286]:
df_total = df.groupby(pd.Grouper(freq='W', key='date')).sum().fillna(0).unstack('date', 0)
df_total.index.levels[1]

len(df_total) == len(df_total.index.levels[1])

trace = go.Scatter(
    x = df_total.index.levels[1],
    y = df_total
)

layout = go.Layout(
    title='Total Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='total-sales')

### New Stores

In [289]:
store_sales = df.groupby(['store']+[pd.Grouper(freq='W', key='date')]).sum().fillna(0).unstack('date')
stores_with_sales = store_sales['sales'].where(store_sales.sales > 0).count()

stores_with_sales.index

trace = go.Bar(
    x = stores_with_sales.index,
    y = stores_with_sales
)

layout = go.Layout(
    title='No. of Stores with Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='stores-with-sales')

### New Items

In [264]:
item_sales = df.groupby(['item']+[pd.Grouper(freq='W', key='date')]).sum().fillna(0).unstack('date')
items_with_sales = item_sales['sales'].where(item_sales.sales > 0).count()

items_with_sales.index

trace = go.Bar(
    x = items_with_sales.index,
    y = items_with_sales
)

layout = go.Layout(
    title='No. of Items with Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='items-with-sales')

## Prepare Data

#### Trim date range to have consistent history

But before we do that, let's trim the range of the data to go from 2016-03-01 to 2016-12-15 in order to avoid anomolies in the data evident in the total sales graph above. We're going to trim to an approximate range here to reduce the amount of data we have to process. Once we aggregate into the order periods we will trim so that we end up with whole periods on either end of our data.

In [290]:
df = df[(df['date'] > '2016-02-15') & (df['date'] < '2016-12-30')]

#### Add records with zero sales so there is a record for every period for every store-item combination

We need to make sure that we add records with zero sales for any combination of store, item and period that doesn't appear in the dataset so that our trailing averages are calculated correctly.

In [291]:
import itertools

beg_date = df['date'].min()
end_date = df['date'].max()
dates = pd.DatetimeIndex(start=beg_date, end=end_date, freq='D')

items = df['item'].value_counts().index
stores = df['store'].value_counts().index
all_periods = pd.DataFrame(list(itertools.product(dates, stores, items)), columns=['date', 'store', 'item'])

all_periods['sales'] = 0
all_periods.date = pd.to_datetime(df.date)
all_periods.item = df.item.astype('category')
all_periods.store = df.store.astype('category')

incl_periods = df.groupby(['store', 'item', 'date']).sum().fillna(0).reset_index()

df_all = pd.concat([all_periods, df]).groupby(['store', 'item', 'date']).sum().fillna(0).reset_index()
len(df_all)

2237703

#### Add Columns for Periods

This time we also want to predict sales for order periods of two and three times per week as well as weekly, assuming orders are placed on the same day each week.

For the **two** orders per week periods, we will predict sales from:
* Tuesday through Thursday (days 2 through 4)
* Friday through Sunday (days 5, 6, 0 and 1)

For the **three** orders per week periods, we will predict sales from:
* Monday through Wednesday (days 1 through 3)
* Thursday and Friday (days 4 and 5)
* Saturday and Sunday (days 6 and 1)

In order to aggregate sales over the correct periods we need to add columns to represent the series for each order period. The end of the two orders per week periods are created in the column `freq2_end` and the three orders per week periods are in `freq3_end`.

We are also adding in columns to distinguish between the intra-weekly periods (which have a zero index) in order to allow the model to compensate for differences in sales volumes between intra-weekly periods, which are `freq2_per` and `freq3_per`.

In [299]:
df_f = df_all.copy()

# Assign each record to its respective intra-week group.
#
# These are the intra-week periods that each day of the week belongs to.
freq = list([[0, 0, 1, 1, 1, 0, 0],
            [0, 1, 1, 1, 2, 2, 0]
            ])

# Map the day of the week of each record to its respective intra-week period.
for i, f in enumerate(freq):
    df_f['freq' + str(i + 2) + '_per'] = df_f['date'].dt.weekday.map(pd.Series(f))

# Assign each record to its respective group within each series of intra-week groups.
# Group membership is indicated in a separate column by the end date of the group.
#
#     1. Calculate the numeric day of the week for each date in the range of dates
#        in the data.
#     2. Create a boolean array with an entry for each record indicating whether
#        the date of the record falls on a day of the week on which an intra-week
#        period ends.
#     3. Calculate the cumulative sum of the boolean area for the range of dates, which
#        will then represent the sequential period each date in our range belongs to.
#     4. Index the cumulative sums by the range of dates to create a lookup table.
#     5. Map the 'date' column in our data to the sequence number using the lookup table.
#     6. Group the lookup table by the period, aggregating the date column by max, which
#        represents the end date of each sequential period, to create another lookup table.
#     7. Map the sequence number series we created earlier to the period ending date using
#        the new lookup table and add it to our data frame.
#

# These are the days of the week that new periods begin on for order frequencies of two
# and three times per week.
period_ends = list([[2, 5],
                   [1, 4, 6]
                   ])

# Execute the same process for each of our order frequencies
for i, p in enumerate(period_ends):
    # Steps 1 through 4
    periods = pd.Series(dates.weekday).isin(period_ends[i]).cumsum()
    date_lookup = pd.DataFrame({'date': dates, 'period': periods})
    date_lookup.set_index('date', inplace=True)
    
    # Step 5
    seq_col = df_f['date'].map(date_lookup.period)

    # Step 6
    period_lookup = date_lookup.reset_index().groupby('period').max()
    period_lookup.to_csv('freq' + str(i+2) + '.csv')

    # Step 7
    df_f['freq' + str(i+2) + '_end'] = seq_col.map(period_lookup.date)

df_f.head()

,store,item,date,sales,freq2_per,freq3_per,freq2_end,freq3_end
0,SEGWD103,41774,2016-02-16,0.0,0,1,2016-02-16,2016-02-18
1,SEGWD103,41774,2016-02-17,1.0,1,1,2016-02-19,2016-02-18
2,SEGWD103,41774,2016-02-18,1.0,1,1,2016-02-19,2016-02-18
3,SEGWD103,41774,2016-02-19,1.0,1,2,2016-02-19,2016-02-20
4,SEGWD103,41774,2016-02-20,0.0,0,2,2016-02-23,2016-02-20


That looks like its working right.

#### Calculating trailing averages
Adding in the rolling average sales is more complicated because we want to calculate the average of like intra-week periods. For example, for the two orders per week frequency the first period spans from Friday through Monday. When we do our trailing averages we want to only include trailing Monday through Friday periods and exclude the Tuesday through Thursday periods that would be included in a strictly sequential calculation. 

In [293]:
# This function calculates the trailing average for a given order frequency. It also
# adds a column for cumulative sales which we user later to filter out records
# for which the store and/or item have no prior sales history.
#
def calc_trailing(orders_per_week):
    # Check to make sure orders per week is in the available range
    if orders_per_week not in [2, 3]:
        print('Orders per week must be either 2 or 3.')
        raise
    
    freq_per = 'freq' + str(orders_per_week) + '_per'
    freq_end = 'freq' + str(orders_per_week) + '_end'
    freq_end_avg = freq_end + '_avg'
    
    f = {'sales': 'sum', freq_per: 'mean'}
    g = ['store', 'item', freq_end]
    
    # Here we filter the data frame for each of the intra-week periods in
    # the specified order frequency and perform the trailing average and
    # calculations on them separately.
    df_final = pd.DataFrame()
    for i, n in enumerate(df_f[freq_per].value_counts().index):
        df_model = df_f[df_f[freq_per] == n].groupby(g).agg(f).fillna(0)
        
        rolling_sum = (df_model
                       .apply(lambda x:x.rolling(window=3).mean())
                       .shift(1)
                      )

        df_model[freq_end_avg] = rolling_sum['sales']
        df_final = df_final.append(df_model.reset_index())
          
    return df_final.groupby(g).sum()

In [370]:
df_final = calc_trailing(2)
df_final.head(10)

freq2_per  sales  freq2_end_avg
store    item  freq2_end                                  
SEGWD103 41774 2016-02-16          0    0.0            NaN
               2016-02-19          1    3.0            NaN
               2016-02-23          0    0.0            NaN
               2016-02-26          1    1.0            NaN
               2016-03-01          0    2.0            NaN
               2016-03-04          1    2.0            NaN
               2016-03-08          0    2.0       0.666667
               2016-03-11          1    3.0       2.000000
               2016-03-15          0    2.0       1.333333
               2016-03-18          1    2.0       2.000000

### Remove Missing Stores and Items

In [371]:
def remove_missing(df_final):
    beg_len = len(df_final)
    df_final['cum_sales'] = df_final.groupby(level=[0]).cumsum()['sales']
    df_final_masked = df_final[df_final.cum_sales != 0].dropna(how='any')
    
    print('{} records removed'.format(beg_len - len(df_final_masked)))
    print('{} records remaining'.format(len(df_final_masked)))
    
    return df_final_masked.reset_index()

In [372]:
df_final_masked = remove_missing(df_final)
df_final_masked.head(10)

9050 records removed
640378 records remaining


,store,item,freq2_end,freq2_per,sales,freq2_end_avg,cum_sales
0,SEGWD103,41774,2016-03-08,0,2.0,0.666667,10.0
1,SEGWD103,41774,2016-03-11,1,3.0,2.000000,13.0
2,SEGWD103,41774,2016-03-15,0,2.0,1.333333,15.0
3,SEGWD103,41774,2016-03-18,1,2.0,2.000000,17.0
4,SEGWD103,41774,2016-03-22,0,4.0,2.000000,21.0
5,SEGWD103,41774,2016-03-25,1,1.0,2.333333,22.0
6,SEGWD103,41774,2016-03-29,0,5.0,2.666667,27.0
7,SEGWD103,41774,2016-04-01,1,1.0,2.000000,28.0
8,SEGWD103,41774,2016-04-05,0,2.0,3.666667,30.0
9,SEGWD103,41774,2016-04-08,1,3.0,1.333333,33.0


### Encode Store and Item Categories

In [373]:
# We have some extra logic here to deal with needing to categorize the three order
# per week variable, but not the two, which is already binary.
def encode_cat_vars(df_final_masked):
  
    # Make sure sales is the first column
    cols = df_final_masked.columns.tolist()
    cols.remove('sales')
    cols = ['sales'] + cols    
    df_final_masked = df_final_masked[cols]
    
    # Initial list of columns to drop
    drop_cols = ['cum_sales', 41793, 'SEGWD103']

    # Initial dummy variables
    stores = pd.get_dummies(df_final_masked['store'])
    items = pd.get_dummies(df_final_masked['item'])
    concat_tables = [df_final_masked, stores, items]
    
    # Create dummy variables for freq3_end if necessary
    try:
        freq = cols[cols.index('freq2_end')][:5]
    except:
        freq = 'freq3'
        freq3_d = pd.get_dummies(df_final_masked['freq3_per'], prefix='freq3_per')
        concat_tables.append(freq3_d)
        drop_cols.append(freq + '_per_2')
       
    # Add dummy variables
    df_final_masked = pd.concat(concat_tables, axis=1)
    
    # Drop columns and return
    return df_final_masked.drop(drop_cols, axis=1).sort_values(freq + '_end')

In [374]:
data = encode_cat_vars(df_final_masked)
data.head()

,sales,store,item,freq2_end,freq2_per,freq2_end_avg,SEGWD104,SEGWD116,SEGWD12,SEGWD123,...,42043,42044,42045,42046,42047,42048,42049,42050,42051,42052
383585,0.0,SEGWD2366,41778,2016-02-16,0,1.666667,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
627189,0.0,SEGWD85,41789,2016-02-16,0,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243870,0.0,SEGWD2244,41800,2016-02-16,0,1.333333,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
477594,0.0,SEGWD28,41789,2016-02-16,0,0.333333,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243778,0.0,SEGWD2244,41799,2016-02-16,0,1.333333,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Split Data for Training and Testing

This time we'll train the model with data through the end of October and then use the remaining data to 
test the model. This is where we also do the final trimming of the data to ensure it includes on whole intra-week periods.

In [375]:
# Values are begining of training, end of training, end of testing and
# represent the end of the respective intra-week oder period.
date_range = dict(
    freq2_end=['2016-03-01', '2016-10-28', '2016-12-23'],
    freq3_end=['2016-03-03', '2016-10-29', '2016-12-24']
)

freq_avg = data.columns.tolist()[5]
freq = data.columns.tolist()[3]
beg_train, end_train, end_test = date_range[freq]

# Add constant
data['const'] = 1

# Normalize average sales variable.
mu = data[freq_avg].mean()
std = data[freq_avg].std()
data[freq_avg] = (data[freq_avg] - mu) / std

data_train = data[data[freq].ge(beg_train) & data[freq].le(end_train)]
data_test = data[data[freq].gt(end_train) & data[freq].le(end_test)]

X_train = data_train.iloc[:,5:].as_matrix()
y_train = data_train.iloc[:,0].as_matrix()

X_test = data_test.iloc[:,5:].as_matrix()
y_test = data_test.iloc[:,0].as_matrix()


## Train Model

This is where we actually train the model. I ran it for 200 iterations - more won't likely increase the predictive power of the model, but there are some other diagnostics we can run to see what other improvements we can make.

In [376]:
from sklearn import linear_model
clf = linear_model.SGDRegressor(n_iter=200)
clf.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=200, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

## Evaluate Model

In [377]:
predict = clf.predict(X_test)
predict_neg = predict < 0
error = y_test - predict
error_neg = predict_neg @ error
np.savetxt('modelparams.csv', clf.predict(np.eye(X_test.shape[1])), delimiter=",")
print('R-squared: {:.{p}f}'.format(clf.score(X_test, y_test), p=4))
print('Total error in sales quantity: {:.{p}f}'.format(sum(error), p=0))
print('Total error as a % of actual: {:.{p}f}%'.format(sum(error) / sum(y_test)*100, p=2))
print('Total error in sales quantity with zero min prediction: {:.{p}f}'.format(sum(error)+error_neg, p=0))
print('Total error as a % of actual with zero min prediction: {:.{p}f}%'.format((sum(error)+error_neg) / sum(y_test)*100, p=2))

R-squared: 0.6704
Total error in sales quantity: 6995
Total error as a % of actual: 4.09%
Total error in sales quantity with zero min prediction: 8321
Total error as a % of actual with zero min prediction: 4.87%


This is still quite a good fit overall. As you would expect, it is not quite as accurate as the model that was based on seven day periods.

In [384]:
data_predict = data_test.iloc[:,:5]
data_predict[freq_avg] = data[freq_avg] * std + mu
data_predict['predict'] = predict
data_predict['variance'] = predict - data_predict['sales']
data_predict.head()

,sales,store,item,freq2_end,freq2_per,freq2_end_avg,predict,variance
548557,0.0,SEGWD658,41783,2016-11-01,0,0.000000,0.318745,0.318745
112520,2.0,SEGWD173,41796,2016-11-01,0,3.666667,3.285899,1.285899
251980,1.0,SEGWD2247,42047,2016-11-01,0,1.000000,0.927180,-0.072820
226892,0.0,SEGWD2229,41804,2016-11-01,0,0.000000,-0.087155,-0.087155
251888,0.0,SEGWD2247,42046,2016-11-01,0,1.333333,1.125886,1.125886


In [380]:
py.iplot(data_predict.groupby(freq).sum()[['sales', 'predict']].iplot(asFigure=True,
                               kind='bar',xTitle='Dates',yTitle='Sales',title='Actual vs. Predicted'))

In [385]:
data_predict.groupby(freq).sum()[['sales', 'predict', 'variance']]

,sales,predict,variance
freq2_end,,,
2016-11-01,11932.0,9713.749332,-2218.250668
2016-11-04,9423.0,8896.994626,-526.005374
2016-11-08,16083.0,11718.156223,-4364.843777
2016-11-11,9254.0,9037.160992,-216.839008
2016-11-15,12190.0,12592.872368,402.872368
2016-11-18,9404.0,9039.111617,-364.888383
2016-11-22,12527.0,12576.152722,49.152722
2016-11-25,8141.0,9197.669593,1056.669593
2016-11-29,9409.0,12741.955878,3332.955878


In [386]:
pivot = pd.pivot_table(data_predict,index=['store','item'],columns=[freq],values=['sales', 'predict', 'variance'],aggfunc=[np.sum])
pivot.head(20)

sum                                              \
                    sales                                               
freq2_end      2016-11-01 2016-11-04 2016-11-08 2016-11-11 2016-11-15   
store    item                                                           
SEGWD103 41774        3.0        3.0        3.0        1.0        1.0   
         41775        2.0        1.0        4.0        2.0        1.0   
         41776        0.0        0.0        0.0        2.0        1.0   
         41777        3.0        2.0        3.0        0.0        3.0   
         41778        2.0        2.0        1.0        1.0        3.0   
         41779        1.0        1.0        2.0        1.0        2.0   
         41780        3.0        3.0        1.0        1.0        1.0   
         41781        0.0        0.0        0.0        0.0        0.0   
         41782        2.0        2.0        4.0        1.0        3.0   
         41783        4.0        3.0        5.0        1.0        4.0   
         41786        0.0        0.0        0.0        0.0        0.0   
         41788        3.0        1.0        4.0        2.0        3.0   
         41789        2.0        1.0        3.0        1.0        4.0   
         41790        2.0        1.0        2.0        1.0        2.0   
         41791        6.0        6.0        2.0        2.0        5.0   
         41792       10.0        3.0        7.0        1.0        6.0   
         41793       12.0        4.0       14.0        4.0        8.0   
         41794        7.0        7.0        8.0        2.0        7.0   
         41795        9.0        4.0       14.0        2.0        8.0   
         41796        1.0        5.0       10.0        1.0        6.0   

                                                                       \
                                                                        
freq2_end      2016-11-18 2016-11-22 2016-11-25 2016-11-29 2016-12-02   
store    item                                                           
SEGWD103 41774        2.0        3.0        3.0        1.0        3.0   
         41775        2.0        1.0        3.0        2.0        1.0   
         41776        0.0        1.0        0.0        0.0        0.0   
         41777        3.0        2.0        0.0        2.0        0.0   
         41778        2.0        3.0        1.0        1.0        1.0   
         41779        2.0        4.0        3.0        3.0        1.0   
         41780        3.0        2.0        4.0        1.0        1.0   
         41781        0.0        0.0        0.0        0.0        0.0   
         41782        1.0        6.0        0.0        3.0        2.0   
         41783        2.0        5.0        2.0        2.0        4.0   
         41786        0.0        0.0        0.0        0.0        0.0   
         41788        1.0        1.0        3.0        3.0        1.0   
         41789        1.0        4.0        0.0        2.0        1.0   
         41790        0.0        4.0        1.0        1.0        1.0   
         41791        2.0        4.0        0.0        0.0        0.0   
         41792        1.0        0.0        0.0        0.0        0.0   
         41793        6.0        7.0        9.0        7.0        8.0   
         41794        5.0        9.0        3.0        6.0       10.0   
         41795        6.0        8.0        9.0        9.0        8.0   
         41796        3.0        3.0        4.0        4.0        4.0   

                  ...                                                  \
                  ...       variance                                    
freq2_end         ...     2016-11-22 2016-11-25 2016-11-29 2016-12-02   
store    item     ...                                                   
SEGWD103 41774    ...      -0.554593  -0.833254   1.445407  -0.833254   
         41775    ...       1.201032  -1.356289  -0.077629   1.201032   
         41776    ...      -0.516758   0.761903   0.761903   0.761903   
     

In [ ]:
pivot.to_csv('pivot_' + freq + '.csv')

## Improvement Opportunities
* Run model diagnostics to understand bias and variance.
* Longer history of sales going back at least another year to be able to factor in normal seasonality.
